<div style='float: right'><img src='pic/satogaeri.png'/></div>
## <div id='satogaeri' />さとがえり

In [ ]:
from itertools import groupby, product, takewhile
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = [list(s) for s in """\
aaaaabbbcd
aeeeabfffd
ghehijjjfd
ghhhikljfm
giiiikljmm
nnopqrrrrs
ntopqruuus
vtoooruwus
vtttxywwwy
vzxxxyyyyy""".split()]
hint = [(0,1,-1),(0,7,2),(0,9,-1),(1,2,2),(1,8,0),(2,6,1),
  (2,7,0),(3,0,1),(3,4,-1),(3,9,2),(4,4,2),(5,0,-1),(5,1,0),
  (5,4,2),(5,8,-1),(6,0,-1),(6,7,1),(7,4,-1),(8,1,2),(9,0,2),
  (9,2,-1),(9,3,1),(9,6,1),(9,7,2),(9,8,3),(9,9,4)]
nw, nh = len(data[0]), len(data)
mx = max(nw, nh) + 1
dc = {(i,j) for i,j,_ in hint}
chk = lambda yx: 0<=yx[0]<nh and 0<=yx[1]<nw and yx not in dc

### 問題
* ○をタテヨコいずれかにまっすぐ移動し、すべての○が、太線で区切られたところ（国）１つにつき１つずつ入るようにします
* ○の中の数字は、移動するマス数を表します
* 数字のない○は何マス移動するか不明（移動しないこともあります）
* ○が移動した跡や、他の○のあるところには○を移動できません

### 変数
* 省略

### 制約
* 省略

In [ ]:
m = LpProblem()
vls = []
vo = [[[] for j in range(nw)] for i in range(nh)]
cc = [[LpAffineExpression() for j in range(nw)] for i in range(nh)]
for y, x, n in hint:
    cands = []
    if n == 0:
        cands.append([(y,x)])
    for p, q in product(range(2), range(2)):
        dy, dx = p - q, p + q - 1
        l = list(takewhile(chk, [(y + dy * k, x + dx * k)
                 for k in range(1, 1 + n if n >= 0 else mx)]))
        if len(l) > 0 and (n < 0 or len(l) == n):
            for k in range(0 if n < 0 else len(l) - 1, len(l)):
                cands.append([(y,x)] + l[:k + 1])
    v = addbinvars(len(cands))
    m += lpSum(v) == 1
    for k, cand in enumerate(cands):
        vls.append((v[k], cand))
        for i, j in cand:
            cc[i][j] += v[k]
        vo[cand[-1][0]][cand[-1][1]].append(v[k])
for i, j in product(range(nh), range(nw)):
    m += cc[i][j] <= 1
for _, lst in groupby(sorted(zip([c for s in data for c in s],
        [j for i in vo for j in i])), lambda x:x[0]):
    m += lpSum(j for i in lst for j in i[1]) == 1
%time m.solve()
for vl, cand in vls:
    if value(vl) > 0.5:
        d = 1 if cand[0][0] == cand[-1][0] else 0
        for i, j in cand[:-1]:
            data[i][j] = '|-'[d]
        data[cand[-1][0]][cand[-1][1]] = '*'
print('\n'.join(''.join(s) for s in data))